In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import ttest_ind, f_oneway
import scipy.stats as stats

import sys
from pathlib import Path

# # Add src directory to the Python path
sys.path.append(str(Path.cwd().parent))

from src.data_analysis.df_dataattribute_analysis import categorize_columns
from src.data_sourcing.import_export_data import get_data, save_data

df = get_data('featured_engineered_testing.csv', 'processed_data')
df


,ATL_OR_DR,CAMPAIGN_TYPE,CHANNEL,COST,FREE_TRIALS,HOLIDAY_FLAG,SEASON,MONTH,ATL_OR_DR_CAMPAIGN_TYPE,ATL_OR_DR_CAMPAIGN_TYPE_CHANNEL,...,LOG_COST,LOG_FREE_TRIALS,LOG_COST_PER_FREE_TRIALS,LOG_COST_BY_FREE_TRIALS,AVG_COST_MONTH,MEDIAN_COST_MONTH,AVG_COST_CAMPAIGN_TYPE_CHANNEL,MEDIAN_COST_CAMPAIGN_TYPE_CHANNEL,AVG_FREE_TRIALS_MONTH,MEDIAN_FREE_TRIALS_MONTH
0,DR - Direct Response,Title,paid social,7784.31,86401.15,False,Summer,June,DR - Direct Response_Title,DR - Direct Response_Title_paid social,...,8.959994,11.366768,0.086265,20.326622,16887.563506,3643.265,14781.334567,3309.50,42865.334723,25054.485
1,DR - Direct Response,Title,app,2474.31,2956.74,False,Summer,August,DR - Direct Response_Title,DR - Direct Response_Title_app,...,7.814121,7.992181,0.608045,15.805560,14172.931604,2955.060,15454.451540,3477.08,43953.768382,28127.390
2,ATL - Above The Line,Title,paid social,10222.82,12513.08,False,Summer,July,ATL - Above The Line_Title,ATL - Above The Line_Title_paid social,...,9.232476,9.434610,0.597171,18.666908,13667.902515,2994.470,14781.334567,3309.50,45959.687637,27091.210
3,DR - Direct Response,Title,app,49631.87,17207.04,False,Winter,December,DR - Direct Response_Title,DR - Direct Response_Title_app,...,10.812409,9.753132,1.356967,20.565462,18937.983302,3240.025,15454.451540,3477.08,47495.480279,28640.060
4,ATL - Above The Line,Title,paid social,2081.06,21758.33,False,Summer,August,ATL - Above The Line_Title,ATL - Above The Line_Title_paid social,...,7.641113,9.987798,0.091343,17.628385,14172.931604,2955.060,14781.334567,3309.50,43953.768382,28127.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14103,ATL - Above The Line,Brand,bvod,1478.78,16941.67,False,Winter,February,ATL - Above The Line_Brand,ATL - Above The Line_Brand_bvod,...,7.299649,9.737591,0.083685,17.036504,13900.634352,3124.380,25653.190823,2983.70,41704.905263,26583.695
14104,ATL - Above The Line,Title,ooh,839.25,5727.03,False,Spring,April,ATL - Above The Line_Title,ATL - Above The Line_Title_ooh,...,6.733699,8.653127,0.136750,15.385461,14257.447211,2830.540,12102.043469,2953.66,46600.296441,28029.530
14105,ATL - Above The Line,Title,bvod,193796.71,34824.51,False,Winter,December,ATL - Above The Line_Title,ATL - Above The Line_Title_bvod,...,12.174570,10.458105,1.881745,22.632642,18937.983302,3240.025,17476.078974,3298.95,47495.480279,28640.060
14106,ATL - Above The Line,Brand,paid social,84690.65,25140.77,False,Winter,February,ATL - Above The Line_Brand,ATL - Above The Line_Brand_paid social,...,11.346772,10.132286,1.474456,21.479007,13900.634352,3124.380,15413.423001,3011.83,41704.905263,26583.695
